In [ ]:
!pip3 install -q langchain
!pip3 install -q huggingface_hub
!pip3 install -q sentence_transformers
!pip3 install -q chromadb

In [ ]:
huggingface_hub.login()

# Loading

In [ ]:
from langchain.document_loaders import TextLoader

In [ ]:
loader = TextLoader("article.txt")
doc = loader.load()

# Splitting

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 600, chunk_overlap = 100)
all_splits = text_splitter.split_documents(doc)

# Storing

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
import huggingface_hub

In [ ]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
    )

In [ ]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=all_splits, embedding=hf)

# Retrieve

In [ ]:
question = "What happened?"
docs = vectorstore.similarity_search(question)
len(docs)

# Generate

In [ ]:
from langchain.llms import HuggingFacePipeline, HuggingFaceHub
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [ ]:
model_id = "meta-llama/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer, max_new_tokens=200
    )
hf = HuggingFacePipeline(pipeline=pipe)

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
qa_chain = RetrievalQA.from_chain_type(hf,retriever=vectorstore.as_retriever())
qa_chain({"query": question})